# Домашнее задание № 2

### Выполнил: Кузнецов С.И. 

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

import pandas as pd

Задание:

* используйте данные MovieLens 1M
* можно использовать любые модели из пакета
* получите RMSE на тестовом сете 0.87 и ниже


Загружаем датасет

In [2]:
ratings = pd.read_csv('./dataset/ratings.dat', sep = ':', usecols = [0, 2, 4, 6],
                      names = ['UserID', 'MovieID', 'Rating', 'Timestamp'])

ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
UserID       1000209 non-null int64
MovieID      1000209 non-null int64
Rating       1000209 non-null int64
Timestamp    1000209 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


In [4]:
movies = pd.read_csv('./dataset/movies.dat', sep = '::',
                     names = ['MovieID', 'Title', 'Genres'])

movies.head()

/home/sergmath/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_with_ratings = movies.join(ratings.set_index('MovieID'), on='MovieID').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,MovieID,Title,Genres,UserID,Rating,Timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,978233496.0
3,1,Toy Story (1995),Animation|Children's|Comedy,9.0,5.0,978225952.0
4,1,Toy Story (1995),Animation|Children's|Comedy,10.0,5.0,978226474.0


Подгружаем данные в библиотеку *surprise*

In [6]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.UserID,
    'iid': movies_with_ratings.Title,
    'rating': movies_with_ratings.Rating
})

In [7]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Строим модель с использованием SVD. Точность проверяем с помощью кросс-валидации.

In [8]:
algo = SVD(n_factors=40, n_epochs=30)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8720  0.8731  0.8718  0.8732  0.8744  0.8729  0.0009  
Fit time          173.92  174.58  174.20  173.20  174.41  174.06  0.49    
Test time         13.14   13.04   12.10   12.98   12.95   12.84   0.38    


{'test_rmse': array([0.87201686, 0.87307076, 0.87178526, 0.87323376, 0.87440932]),
 'fit_time': (173.91785550117493,
  174.58455896377563,
  174.19917607307434,
  173.19682693481445,
  174.40980577468872),
 'test_time': (13.136953115463257,
  13.041775465011597,
  12.100778818130493,
  12.979237794876099,
  12.947546482086182)}

Как видим, точность RMSE > 0.87 достигается на каждом фолде.